In [1]:
frequencies = {
        "a": 0.08167,
        "b": 0.01492,
        "c": 0.02782,
        "d": 0.04253,
        "e": 0.12702,
        "f": 0.02228,
        "g": 0.02015,
        "h": 0.06094,
        "i": 0.06966,
        "j": 0.00153,
        "k": 0.00772,
        "l": 0.04025,
        "m": 0.02406,
        "n": 0.06749,
        "o": 0.07507,
        "p": 0.01929,
        "q": 0.00095,
        "r": 0.05987,
        "s": 0.06327,
        "t": 0.09056,
        "u": 0.02758,
        "v": 0.00978,
        "w": 0.02360,
        "x": 0.00150,
        "y": 0.01974,
        "z": 0.00074,
    }
def caesar_decrypt(ciphertext):
    alphabet = [chr(i) for i in range(97, 123)]
    # Chi squared statistic values
    chi_squared_values = {}
 
    # cycle through all of the shifts
    for shift in range(len(alphabet)):
        decrypted_with_shift = ""
 
        # decrypt the message with the shift
        for letter in ciphertext:
            # Try to index the letter in the alphabet
            new_key = (alphabet.index(letter) - shift) % len(
                alphabet
            )
            decrypted_with_shift += alphabet[new_key]
 
        chi_squared = 0.0
 
        # Loop through each letter in the decoded message with the shift
        for letter in decrypted_with_shift:
              if letter in frequencies:
                  # Get the amount of times the letter occurs in the message
                  occurrences = decrypted_with_shift.count(letter)

                  # Get the excepcted amount of times the letter should appear based
                  # on letter frequencies
                  expected = frequencies[letter] * occurrences

                  # Complete the chi squared statistic formula
                  chi_letter_value = ((occurrences - expected) ** 2) / expected

                  # Add the margin of error to the total chi squared statistic
                  chi_squared += chi_letter_value
 
        # Add the data to the chi_squared_values dictionary
        chi_squared_values[shift] = [
            chi_squared,
            decrypted_with_shift,
        ]
 
    # Get the most likely cipher by finding the cipher with the smallest chi squared
    # statistic
    most_likely_cipher = min(
        chi_squared_values, key=chi_squared_values.get
    )
    # Get all the data from the most likely cipher (key, decoded message)
    most_likely_cipher_chi_squared_value = chi_squared_values[
        most_likely_cipher
    ][0]
    decoded_most_likely_cipher = chi_squared_values[most_likely_cipher][1]

    # Return the data on the most likely shift
    return (
        most_likely_cipher,
        decoded_most_likely_cipher
    )

In [2]:
EN_IC = sum([a*a for a in frequencies.values()])
# Adds every sizeth character in one string and returns it
def get_group(s, size, offset=0):
    group = ""
    for i in range(0,len(s)-offset,size):
        group += s[i+offset]
    return group

# Returns a list and a dictionay of every letter's frequency of a text
def get_letters_freq(s):
    s = s.lower()
    letters_freq_dict = {chr(i):0 for i in range(97, 123)}
    for l in s:
        letters_freq_dict[l] += 1
    letters_freq_list = [a for a in letters_freq_dict.values()]
    return letters_freq_list

# Returns the IC of a piece of text
def _ic(letters_freq_list):
    numerator = sum(freq*(freq-1) for freq in letters_freq_list)
    text_size = sum(freq for freq in letters_freq_list)
    denominator = text_size*(text_size-1)

    return float(numerator/float(denominator))

# Returns the most probably key length for the ciphertext
def find_key_length(cyphertext, max_key_len):
    min_diff = float('inf')
    key_len = 0
    for candidate_length in range(1, max_key_len + 1):
        group = get_group(cyphertext, candidate_length)
        freq_list = get_letters_freq(group)
        ic = _ic(freq_list)
        if EN_IC-ic < min_diff:
            min_diff = EN_IC-ic
            key_len = candidate_length
    return key_len

# Returns the most probably key given key length and cipher
def get_key(cipher, key_length):
    key = ""
    for i in range(key_length):
      # Using caesar decrypt per group
      group = get_group(cipher, key_length, i)
      key_frag,_= caesar_decrypt(group)
      key+=(chr(key_frag+ord('a')))
    key_final=key
    key_final*=(int(len(cipher)/len(key))+1)
    return key,key_final

def vigenere_decrypt(ciphertext):
    orig_text = ""
    key_len = find_key_length(ciphertext, 10)
    key, key_final = get_key(ciphertext,key_len)
    for i in range(len(ciphertext)):
        # Shifting each letter according to key
        x = (ord(ciphertext[i])-ord(key_final[i]) + 26) % 26
        x += ord('a') 
        orig_text+=chr(x)
    return key, orig_text

In [3]:
monoalphabetic_ic = 0.065
polyalphabetic_ic = 0.038

def decypher(ciphertext):
    print(f"Ciphertext: {ciphertext}")
    # Remove spaces and change to lowercase
    processed_ciphertext=''.join(ciphertext.split())
    # Determine cipher:
    if is_playfair(processed_ciphertext):
        #key, plaintext=playfair_decrypt(processed_ciphertext)
        print('###PLAYFAIR CIPHER###')
        !gcc -O3 playfaircrack.c -lm -o pfc && ./pfc KCWSGURERSFZMOFMFYBKMTGNITBMDQFELHWCVGLBAQQAHPGKAZBSAKTMVSHADTRSQKKBTGOVMUHNTGXROUGLUHQBFAPYPGQLTMFLRMAQPOCRPGQLFYFMAEQKRFADPGPOECGMMXBPLYNSIKFRGNU
    else:
        processed_ciphertext=processed_ciphertext.lower()
        processed_freq_list = get_letters_freq(processed_ciphertext)
        processed_ic = _ic(processed_freq_list)
        if abs(processed_ic - monoalphabetic_ic) < abs(processed_ic - polyalphabetic_ic):
            print('###CAESAR CIPHER###')
            key, plaintext=caesar_decrypt(processed_ciphertext)
            plaintext_list = list(plaintext)
            for i, l in enumerate(ciphertext):
                if l == ' ':
                    plaintext_list.insert(i, ' ')
            plaintext=''.join(plaintext_list)
            print(f"key: {key}\nplaintext: {plaintext}")
        else:
            print('###VIGENERE###')
            key, plaintext=vigenere_decrypt(processed_ciphertext)
            plaintext_list = list(plaintext)
            for i, l in enumerate(ciphertext):
                if l == ' ':
                    plaintext_list.insert(i, ' ')
            plaintext=''.join(plaintext_list)
            print(f"key: {key}\nplaintext: {plaintext}")


In [4]:
def is_playfair(ciphertext):
    playfair=True
    for i in range(0, len(ciphertext), 2):
        if(ciphertext[i]==ciphertext[i+1]):
            playfair=False
            break
    return playfair

In [5]:
ciphertexts = [
    "Fubswrjudskb lv d idvflqdwlqj ilhog dqg surylghv vhfxulwb iru pdqb ri wkh dssolfdwlrqv wkdw zh xvh lq wrgdbv zruog dqg pdnhv rxu zruog d vdihu sodfh iru rxu gdlob frppxqlfdwlrq",
    "Eyyizdiyainn kz a yghepntzxpn fbkaf hnw vgqciwkh ulcnxxvf fhx bcuy hl ijl aivakjamodpz tagi yl ulk xp aowgnu dokrs cud fgzgz onx lqylw g hcmek vacje yug qbr wgxnf chsbwuivgikvn",
    "KCWSGURERSFZ MO F MFYBKMTGNIT BMDQF ELH WCVGLBAQ QAHPGKAZ BSA KTMV SH ADT RSQKKBTGOVMU HNTG XR OUG LU HQBFAP YPGQL TMF LRMAQ POC RPGQL F YFMAE QKRFA DPG POE CGMMX BPLYNSIKFRGNUI"
]
for ciphertext in ciphertexts:
  decypher(ciphertext)

Ciphertext: Fubswrjudskb lv d idvflqdwlqj ilhog dqg surylghv vhfxulwb iru pdqb ri wkh dssolfdwlrqv wkdw zh xvh lq wrgdbv zruog dqg pdnhv rxu zruog d vdihu sodfh iru rxu gdlob frppxqlfdwlrq
###CAESAR CIPHER###
key: 3
plaintext: cryptography is a fascinating field and provides security for many of the applications that we use in todays world and makes our world a safer place for our daily communication
Ciphertext: Eyyizdiyainn kz a yghepntzxpn fbkaf hnw vgqciwkh ulcnxxvf fhx bcuy hl ijl aivakjamodpz tagi yl ulk xp aowgnu dokrs cud fgzgz onx lqylw g hcmek vacje yug qbr wgxnf chsbwuivgikvn
###VIGENERE###
key: chatgp
plaintext: cryptography is a fascinating field and provides security for many of the applications that we use in todays world and makes our world a safer place for our daily communication
Ciphertext: KCWSGURERSFZ MO F MFYBKMTGNIT BMDQF ELH WCVGLBAQ QAHPGKAZ BSA KTMV SH ADT RSQKKBTGOVMU HNTG XR OUG LU HQBFAP YPGQL TMF LRMAQ POC RPGQL F YFMAE QKRFA DPG POE CGMMX BPLYNSIKFRGNUI
##